## GRV API

- acc email: maxharper26@icloud.com
- acc password: kowGot-sonbyk-4fazko
- api key: 4157732f-fa7f-4459-ab26-089e4deab17a
- documentation: https://topaz.grv.org.au/docs/#/codes/GetTrack

In [1]:
import pandas as pd
from topaz import TopazAPI
import time
import requests
from requests.exceptions import HTTPError
from json import JSONDecodeError
import numpy as np

In [2]:
key = '4157732f-fa7f-4459-ab26-089e4deab17a'
key_m = '6a8ffdc7-ee7a-4795-9b98-607e3e838342'
key_p ='34b33827-3ca1-4c2b-a4c5-40fde1d99a84'


In [3]:
topaz_api = TopazAPI(key)

In [4]:
authority_codes = [ 'NSW', 'QLD', 'SA', 'TAS', 'VIC', 'WA','NT']
all_races = []
missed_run=False

for code in authority_codes:
    try:
        races = topaz_api.get_races(from_date='2024-03-17', to_date='2024-03-18', owning_authority_code=code)
        all_races.append(races)
    except Exception as e:
        print(f"Error fetching races for {code}: {e}")

all_races_df = pd.concat(all_races, ignore_index=True)
race_ids = list(all_races_df['raceId'].unique())
race_ids

[1006994033,
 1006994034,
 1006994035,
 1006994036,
 1006994037,
 1006994038,
 1006994039,
 1006994040,
 1006994041,
 1006994042,
 1007029048,
 1007029049,
 1007029050,
 1007029051,
 1007029052,
 1007029053,
 1007029054,
 1007029055,
 1007029056,
 1007029057,
 1007029058,
 1007029059,
 1007029060,
 1007029061,
 1007029062,
 1007029063,
 1007029064,
 1007029065,
 1007029066,
 1007029067,
 1007029068,
 1007461416,
 1007461419,
 1007461422,
 1007461425,
 1007461428,
 1007461431,
 1007461434,
 1007461437,
 1007461442,
 1007461445,
 1007461417,
 1007461420,
 1007461423,
 1007461426,
 1007461429,
 1007461432,
 1007461435,
 1007461438,
 1007461440,
 1007461443,
 1007461446,
 1007461448,
 1007461418,
 1007461421,
 1007461424,
 1007461427,
 1007461430,
 1007461433,
 1007461436,
 1007461439,
 1007461441,
 1007461444,
 1007461447,
 1007504487,
 1007504488,
 1007504489,
 1007504490,
 1007504491,
 1007504492,
 1007504493,
 1007504494,
 1007504495,
 1007504496,
 1007062625,
 1007062626,
 1007062628,

In [5]:
len(race_ids)

272

In [15]:
race_ids = pd.read_csv('api_files/missing.csv')['missed']
missed_run=True
race_ids

0       990395989
1       990395990
2       990395991
3       990395992
4       990395993
          ...    
386    1007062655
387    1007062656
388    1007062657
389    1006994012
390    1006994013
Name: missed, Length: 391, dtype: int64

In [16]:
big = ''
start = time.time()
first=0
headers={'X-API-Key':key}
missed=[]



for i in range(first,len(race_ids)):
    rID = race_ids[i]
    url = f'https://topaz.grv.org.au/api/race/{rID}/result'
    try:
        race = requests.get(url.format(rID),headers=headers).json()
    except JSONDecodeError as e:
        race={'error':1,'message':'empty json'}
    
    if('error' in race and 'message' in race):
        print(rID,race['message'])
        missed.append(rID)
    else:
        splits = race['splitTimes']
        
        
        summary = pd.DataFrame(race['runs'])

        cols=['track','distance','raceId','weightInKg','meetingDate','dogName','resultTime','bestFinishTrackAndDistance', 'pir','incomingGrade', 'outgoingGrade', 'rating','resultMarginLengths', 'startPaceCode', 'jumpCode',
           'startPrice','boxNumber']
        summary=summary[cols]
        summary['Plc'] = summary['resultTime'].rank(method='min')

        if(splits!=[]):
            splits = pd.DataFrame(splits)
            splits = splits[['dogName','splitTimeMarker','time']]

            markers = max(splits['splitTimeMarker'])
            for marker in range(1,markers+1):
                split2 = splits[splits['splitTimeMarker']==marker]
                split2=split2.rename(columns={'time':'split_'+str(marker)})[['dogName','split_'+str(marker)]]
                summary = pd.merge(summary,split2,on='dogName',how='outer')


        if(type(big)==str):
            big = summary
        else:
            big = pd.concat([big,summary])

            



end = time.time()
taken = end-start
print('time for',len(race_ids),':',round(taken,3),'avg:',round(taken/len(race_ids),2))


990395989 Race does not yet have results.
990395990 Race does not yet have results.
990395991 Race does not yet have results.
990395992 Race does not yet have results.
990395993 Race does not yet have results.
990395994 Race does not yet have results.
990395995 Race does not yet have results.
990395996 Race does not yet have results.
990395997 Race does not yet have results.
990395998 Race does not yet have results.
990395999 Race does not yet have results.
990376520 Race does not yet have results.
990376521 Race does not yet have results.
990376522 Race does not yet have results.
990376523 Race does not yet have results.
990376524 Race does not yet have results.
990376525 Race does not yet have results.
990376526 Race does not yet have results.
990376527 Race does not yet have results.
990376528 Race does not yet have results.
990376529 Race does not yet have results.
990376530 Race does not yet have results.
990376531 Race does not yet have results.
990376739 Race does not yet have r

1007461456 empty json
1007461457 empty json
1007461458 empty json
1007461459 empty json
1007831149 empty json
1007831139 empty json
1007831143 empty json
1007831140 empty json
1007831141 empty json
1007831142 empty json
1007831145 empty json
1007062646 empty json
1007062647 empty json
1007062648 empty json
1007062649 empty json
1007062650 empty json
1007062651 empty json
1007062652 empty json
1007062653 empty json
1007062654 empty json
1007062655 empty json
1007062656 empty json
1007062657 empty json
1006994012 empty json
1006994013 empty json
time for 391 : 109.886 avg: 0.28


In [17]:
big

,track,distance,raceId,weightInKg,meetingDate,dogName,resultTime,bestFinishTrackAndDistance,pir,incomingGrade,...,rating,resultMarginLengths,startPaceCode,jumpCode,startPrice,boxNumber,Plc,split_1,split_2,split_3
0,Bundaberg,315,1007504500,NaN,2024-03-18T00:00:00.000Z,LONDON STAR,NaN,NaN,None,7,...,100,None,None,None,1.0,NaN,NaN,NaN,NaN,NaN
1,Bundaberg,315,1007504500,23.7,2024-03-18T00:00:00.000Z,TINY TURNIPS,19.280,19.150,None,Maiden,...,0,12.25L,None,None,26.0,1.0,8.0,NaN,NaN,NaN
2,Bundaberg,315,1007504500,29.5,2024-03-18T00:00:00.000Z,MIA JET,18.490,18.490,None,Maiden,...,73,0.50L,None,None,4.4,2.0,2.0,NaN,NaN,NaN
3,Bundaberg,315,1007504500,32.8,2024-03-18T00:00:00.000Z,BABY NO GO,18.520,18.450,None,Maiden,...,83,1.00L,None,None,6.0,3.0,3.0,NaN,NaN,NaN
4,Bundaberg,315,1007504500,23.8,2024-03-18T00:00:00.000Z,LOYAL LIES,19.230,19.230,None,Maiden,...,0,11.25L,None,None,12.0,4.0,7.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,Shepparton,390,1009058595,30.2,2024-03-21T00:00:00.000Z,HOLLYWOOD FRANKY,22.274,22.274,11,6,...,96,3.00L,None,Medium,4.9,6.0,1.0,8.49000,NaN,NaN
6,Shepparton,390,1009058595,28.1,2024-03-21T00:00:00.000Z,AILEEN'S LAD,22.609,22.522,43,6,...,96,5.00L,None,Slow,12.8,7.0,3.0,8.75425,NaN,NaN
7,Shepparton,390,1009058595,29.6,2024-03-21T00:00:00.000Z,SPIRIT OF PANAMA,22.748,22.585,24,6,...,92,7.25L,None,Medium,13.8,8.0,4.0,8.68250,NaN,NaN
8,Shepparton,390,1009058595,NaN,2024-03-21T00:00:00.000Z,SANDY CABANG,NaN,22.416,None,6,...,93,None,None,None,0.0,NaN,NaN,NaN,NaN,NaN


In [18]:
big1 = big.copy()
big1['Track']=big1['track'].str.lower()
big1['dogName']=big1['dogName'].str.lower()

to_rename={
    'Plc':'Place',
    'dogName':'DogName',
    'boxNumber':'Box',
    'resultTime':'RunTime',
    'weightInKg':'Weight',
    'startPrice':'StartPrice',
    'pir':'PIR',
    'distance':'Distance',
    'raceId':'RaceId',
    'incomingGrade':'RaceGrade',
    'meetingDate':'date',
    'split_1':'Split'
}

big1=big1.rename(columns=to_rename)
big1 = big1[['Place','DogName','Box','Weight','StartPrice','PIR','Split','RunTime','RaceId','Distance','RaceGrade','Track','date']]
big1['TrainerLName']='poo'
big1['TrainerFName']='poo'

big1['date']=big1['date'].astype(str).str.split('T').str[0]
big1['date']=pd.to_datetime(big1['date'])


import os
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
m=0
print(files)
for file in files:
    num = file.split('_')[1].split('.')[0]
    num=int(num)
    if(num>m):
        m=num
print(m)

big1.to_csv('api_files/api_'+str(m+1)+'.csv',index=False)
big1

['api_8.csv', 'api_9.csv', 'api_17.csv', 'api_16.csv', 'api_14.csv', 'api_28.csv', 'api_29.csv', 'api_15.csv', 'api_39.csv', 'api_11.csv', 'api_10.csv', 'api_38.csv', 'api_12.csv', 'api_13.csv', 'api_36.csv', 'api_22.csv', 'api_23.csv', 'api_37.csv', 'api_21.csv', 'api_35.csv', 'api_34.csv', 'api_20.csv', 'api_18.csv', 'api_24.csv', 'api_30.csv', 'api_31.csv', 'api_25.csv', 'api_19.csv', 'api_33.csv', 'api_27.csv', 'api_26.csv', 'api_32.csv', 'api_7.csv', 'api_41.csv', 'api_40.csv', 'api_6.csv', 'api_4.csv', 'api_42.csv', 'api_43.csv', 'api_5.csv', 'api_1.csv', 'api_2.csv', 'api_44.csv', 'api_3.csv']
44


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,NaN,london star,NaN,NaN,1.0,None,NaN,NaN,1007504500,315,7,bundaberg,2024-03-18,poo,poo
1,8.0,tiny turnips,1.0,23.7,26.0,None,NaN,19.280,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
2,2.0,mia jet,2.0,29.5,4.4,None,NaN,18.490,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
3,3.0,baby no go,3.0,32.8,6.0,None,NaN,18.520,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
4,7.0,loyal lies,4.0,23.8,12.0,None,NaN,19.230,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1.0,hollywood franky,6.0,30.2,4.9,11,8.49000,22.274,1009058595,390,6,shepparton,2024-03-21,poo,poo
6,3.0,aileen's lad,7.0,28.1,12.8,43,8.75425,22.609,1009058595,390,6,shepparton,2024-03-21,poo,poo
7,4.0,spirit of panama,8.0,29.6,13.8,24,8.68250,22.748,1009058595,390,6,shepparton,2024-03-21,poo,poo
8,NaN,sandy cabang,NaN,NaN,0.0,None,NaN,NaN,1009058595,390,6,shepparton,2024-03-21,poo,poo


In [19]:
if(missed_run==False):
    missing  = pd.read_csv('api_files/missing.csv')[['missed']]
    missing2 = pd.DataFrame(missed,columns=['missed'])
    missing3 = pd.concat([missing,missing2])
    missing4 = missing3['missed'].unique()
    missing5 = pd.DataFrame(missing4,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
else:
    missing5 = pd.DataFrame(missed,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
    
missing5

,missed
0,990395989
1,990395990
2,990395991
3,990395992
4,990395993
...,...
294,1007062655
295,1007062656
296,1007062657
297,1006994012


In [20]:
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
files.sort()
files
merged = pd.read_csv('api_files/'+files[0])
for file in files[1:]:
    merged=pd.concat([merged,pd.read_csv('api_files/'+file)])

merged=merged.sort_values(by='date')
merged['Track']=np.where(merged['Track'].str.contains('dport @'),'hobart',merged['Track'])
merged

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,6.0,chick magnet,1.0,30.5,5.5,566.0,NaN,19.700,990412069,336,5,muswellbrook,2024-01-28,poo,poo
1331,1.0,monkey's style,7.0,34.5,4.9,111.0,5.1800,24.282,992122290,435,5,sale,2024-01-28,poo,poo
1330,3.0,rumour confirmed,6.0,31.9,38.7,743.0,5.4425,24.537,992122290,435,5,sale,2024-01-28,poo,poo
1329,8.0,crew,5.0,34.5,57.1,678.0,5.4075,25.242,992122290,435,5,sale,2024-01-28,poo,poo
1328,7.0,persuasion,4.0,27.1,20.2,887.0,5.4775,25.181,992122290,435,5,sale,2024-01-28,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,4.0,marionvale gal,1.0,26.6,4.4,NaN,11.1000,23.680,1008720936,400,6,mount gambier,2024-03-21,poo,poo
397,NaN,quantum leap,NaN,NaN,1.0,NaN,NaN,NaN,1008720935,400,6,mount gambier,2024-03-21,poo,poo
396,NaN,bella prue,NaN,NaN,1.0,NaN,NaN,NaN,1008720935,400,6,mount gambier,2024-03-21,poo,poo
394,4.0,rise as one,7.0,32.2,8.0,NaN,11.5900,24.520,1008720935,400,6,mount gambier,2024-03-21,poo,poo


In [21]:
a = pd.read_csv('../allRaces2.csv')
b = merged.copy()
c = pd.concat([a,b])
#c = c.drop(columns=['Unnamed: 0.1'])
c = c.drop_duplicates(subset=['DogName','RunTime','date'],keep='first')

c['Track']=np.where(c['Track'].str.contains('murray'),'murray bridge',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])



c.to_csv('../allRaces1.csv',index=False)
c

/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_73790/609129207.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_73790/609129207.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_73790/609129207.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,raine allen,1.0,27.4,2.4,111,6.68,28.66,335811282,500,Restricted Win,bendigo,2018-07-01,Grenfell,C
1,2.0,surf a lot,2.0,32.8,6.3,332,7.01,28.82,335811282,500,Restricted Win,bendigo,2018-07-01,Tyley,C
2,3.0,pingin' bee,6.0,25.5,9.3,443,7.24,28.93,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
3,4.0,lucas the great,7.0,32.2,9.1,655,7.31,29.03,335811282,500,Restricted Win,bendigo,2018-07-01,Hamilton,E
4,5.0,quavo,4.0,28.9,3.4,766,7.38,29.05,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,4.0,marionvale gal,1.0,26.6,4.4,NaN,11.10,23.68,1008720936,400,6,mount gambier,2024-03-21,poo,poo
397,NaN,quantum leap,NaN,NaN,1.0,NaN,NaN,NaN,1008720935,400,6,mount gambier,2024-03-21,poo,poo
396,NaN,bella prue,NaN,NaN,1.0,NaN,NaN,NaN,1008720935,400,6,mount gambier,2024-03-21,poo,poo
394,4.0,rise as one,7.0,32.2,8.0,NaN,11.59,24.52,1008720935,400,6,mount gambier,2024-03-21,poo,poo


In [22]:
## still missing 
for rID in missing5['missed']:
    url = f'https://topaz.grv.org.au/api/race/{rID}/runs'
    try:
        race = requests.get(url.format(rID),headers=headers).json()[0]
        print(race['track'],race['meetingDate'],race['distance'])
    except JSONDecodeError as e:
        print(rID,'fucked')
        
# race

Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 498
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murr

Cannington 2024-03-18T00:00:00.000Z 275
Cannington 2024-03-18T00:00:00.000Z 275
Cannington 2024-03-18T00:00:00.000Z 275
Darwin 2024-03-17T00:00:00.000Z 312
Darwin 2024-03-17T00:00:00.000Z 312


In [23]:
big1

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,NaN,london star,NaN,NaN,1.0,None,NaN,NaN,1007504500,315,7,bundaberg,2024-03-18,poo,poo
1,8.0,tiny turnips,1.0,23.7,26.0,None,NaN,19.280,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
2,2.0,mia jet,2.0,29.5,4.4,None,NaN,18.490,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
3,3.0,baby no go,3.0,32.8,6.0,None,NaN,18.520,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
4,7.0,loyal lies,4.0,23.8,12.0,None,NaN,19.230,1007504500,315,Maiden,bundaberg,2024-03-18,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1.0,hollywood franky,6.0,30.2,4.9,11,8.49000,22.274,1009058595,390,6,shepparton,2024-03-21,poo,poo
6,3.0,aileen's lad,7.0,28.1,12.8,43,8.75425,22.609,1009058595,390,6,shepparton,2024-03-21,poo,poo
7,4.0,spirit of panama,8.0,29.6,13.8,24,8.68250,22.748,1009058595,390,6,shepparton,2024-03-21,poo,poo
8,NaN,sandy cabang,NaN,NaN,0.0,None,NaN,NaN,1009058595,390,6,shepparton,2024-03-21,poo,poo


In [24]:
big1[big1['Track']=='rockhampton']

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,tumbleweeds,1.0,31.1,3.2,None,9.05,24.12,1007920317,407,7,rockhampton,2024-03-20,poo,poo
1,5.0,bahama blue,2.0,31.2,4.0,None,9.18,24.26,1007920317,407,Maiden,rockhampton,2024-03-20,poo,poo
2,3.0,tigger,3.0,23.6,18.0,None,9.32,24.24,1007920317,407,Maiden,rockhampton,2024-03-20,poo,poo
3,7.0,early supper,4.0,28.5,4.8,None,9.42,24.53,1007920317,407,Maiden,rockhampton,2024-03-20,poo,poo
4,1.0,chipped penny,5.0,26.6,7.0,None,9.09,24.12,1007920317,407,Maiden,rockhampton,2024-03-20,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,6.0,sayaka,4.0,31.5,26.0,None,9.53,24.42,1007920328,407,5,rockhampton,2024-03-20,poo,poo
5,1.0,shell and sea,5.0,25.3,5.0,None,9.13,23.81,1007920328,407,5,rockhampton,2024-03-20,poo,poo
6,2.0,enrique fernando,6.0,29.0,6.5,None,9.17,23.85,1007920328,407,5,rockhampton,2024-03-20,poo,poo
7,4.0,dynamic one,7.0,34.6,14.0,None,9.41,24.00,1007920328,407,5,rockhampton,2024-03-20,poo,poo
